In [ ]:
import os
import shutil
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.optimizers.schedules import ExponentialDecay
from keras.applications import ResNet50, InceptionV3, InceptionResNetV2, DenseNet121
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy
import numpy as np
from keras.models import Model
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from google.colab import drive



drive.mount('/content/drive')


In [32]:
# Parameters and hyperparemeters
batch_size = 32
in_epochs = 200
fine_epochs = 50
total_epochs = in_epochs + fine_epochs

# ReduceLROnPlateau
min_lr = 1e-10
factor = 0.2

#Optimizer 1
learning_rate = 0.0001

#Optimizer 2
learning_rate2 = 0.00001

# Early Stopping & Scheduler
patience = 10

# Early Stopping
start_epoch = 50
min_delta= 0.0001

#Layers to unfreeze
layers = 10
# Classes
class_names = ['front_bunched', 'front_up', 'mid_bunched', 'tip_up']
num_classes = len(class_names)

#Image Size
image_size = (256,256)

# Directories
data_dir = '/content/drive/MyDrive/EM401/Class_weights/Dataset'
train_dir = '/content/drive/MyDrive/EM401/Class_weights/train'
val_dir = '/content/drive/MyDrive/EM401/Class_weights/validation'
test_dir = '/content/drive/MyDrive/EM401/Class_weights/test'

# Create directories for train, validation, and test sets
for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

In [ ]:

def count_images_per_class(data_dir):
    class_counts = {}
    total_images = 0

    # Iterate over the subfolders in the main data directory
    for image_class in os.listdir(data_dir):
        # Check if the current item is a directory
        if os.path.isdir(os.path.join(data_dir, image_class)):
            # Count the number of images in the class
            num_images = len(os.listdir(os.path.join(data_dir, image_class)))
            # Store the count for the class
            class_counts[image_class] = num_images
            # Increment total_images count
            total_images += num_images

    return class_counts, total_images

# Call the function to count images per class
class_counts, total_images = count_images_per_class(data_dir)

# Print the counts per class
print("Number of images per class:")
for image_class, count in class_counts.items():
    print(f"{image_class}: {count}")

# Print the total number of images
print("\nTotal number of images:", total_images)

In [ ]:
# @title Example of Calcualtion for Class weights (not needed)

# Number of images per class
num_front_bunched = 121
num_front_up = 95
num_tip_up = 114
num_mid_bunched = 164

# Total number of images
total_images = num_front_bunched + num_front_up + num_tip_up + num_mid_bunched

# Calculate class weights
weight_front_bunched = total_images / num_front_bunched
weight_front_up = total_images / num_front_up
weight_tip_up = total_images / num_tip_up
weight_mid_bunched = total_images / num_mid_bunched

# Normalize weights
total_weight = weight_front_bunched + weight_front_up + weight_tip_up + weight_mid_bunched
weight_front_bunched /= total_weight
weight_front_up /= total_weight
weight_tip_up /= total_weight
weight_mid_bunched /= total_weight


print("Class Weights:")
print("front_bunched:", weight_front_bunched)
print("front_up:", weight_front_up)
print("tip_up:", weight_tip_up)
print("mid_bunched:", weight_mid_bunched)



In [ ]:
# @title Run once
for image_class in os.listdir(data_dir):
    class_train_dir = os.path.join(train_dir, image_class)
    class_val_dir = os.path.join(val_dir, image_class)
    class_test_dir = os.path.join(test_dir, image_class)
    os.makedirs(class_train_dir, exist_ok=True)
    os.makedirs(class_val_dir, exist_ok=True)
    os.makedirs(class_test_dir, exist_ok=True)

    images = os.listdir(os.path.join(data_dir, image_class))
    train_size = int(len(images) * 0.7)
    val_size = int(len(images) * 0.2)
    train_images = images[:train_size]
    val_images = images[train_size:train_size + val_size]
    test_images = images[train_size + val_size:]

    for image in train_images:
        shutil.copy2(os.path.join(data_dir, image_class, image), os.path.join(class_train_dir, image))

    for image in val_images:
        shutil.copy2(os.path.join(data_dir, image_class, image), os.path.join(class_val_dir, image))

    for image in test_images:
        shutil.copy2(os.path.join(data_dir, image_class, image), os.path.join(class_test_dir, image))


In [ ]:
# Assuming normalization is the main preprocessing step now
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],
    rescale=1./255  # Ensuring normalization
)

# For validation and test sets, apply only rescaling for normalization
test_val_datagen = ImageDataGenerator(rescale=1./255)

# Setup generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='sparse'  # Assuming your labels are integer class indices
)

validation_generator = test_val_datagen.flow_from_directory(
    val_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False  # Important for test set to evaluate in order
)

In [ ]:
# Extract class labels from the training generator
classes = train_generator.classes

# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(classes),
    y=classes)

class_weight_dict = dict(enumerate(class_weights))
print("Class Weights:", class_weights)


In [7]:
# Load base model
wResNet50 = '/content/drive/MyDrive/EM401/models/RadImageNet_models/RadImageNet-ResNet50_notop.h5'
wIRV2 = '/content/drive/MyDrive/EM401/models/RadImageNet_models/RadImageNet-IRV2_notop.h5'
wInceptionV3 = '/content/drive/MyDrive/EM401/models/RadImageNet_models/RadImageNet-InceptionV3_notop.h5'
wDenseNet121 = '/content/drive/MyDrive/EM401/models/RadImageNet_models/RadImageNet-DenseNet121_notop.h5'

base_model = InceptionV3(weights= wInceptionV3, include_top=False, input_shape=(256, 256, 3))

for layer in base_model.layers:
    layer.trainable = False

# Create the model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # More units in the Dense layer
x = Dropout(0.5)(x)  # Adjusted dropout rate
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Learning Rate scheduler 1
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience//2, min_lr=min_lr, verbose=1)

In [9]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Set up callbacks
logdir = '/content/drive/MyDrive/EM401/logs'
os.makedirs(logdir, exist_ok=True)
tensorboard_callback = TensorBoard(log_dir=logdir)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience, verbose=1, restore_best_weights=True, start_from_epoch=start_epoch)


# Train model
history = model.fit(
    train_generator,
    epochs=in_epochs,
    validation_data=validation_generator,
    class_weight=class_weight_dict,
    callbacks=[tensorboard_callback, early_stopping, reduce_lr]

)


In [ ]:
# For fine-tuning, unfreeze the last set of layers and recompile the model
for layer in base_model.layers[-layers:]:  # Experiment with the exact number of layers to unfreeze
    layer.trainable = True

# Switch to SGD with a low learning rate for fine-tuning
from keras.optimizers import SGD, AdamW


model.compile(optimizer=SGD(learning_rate=learning_rate2),  # Lower learning rate for fine-tuning
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model with fine-tuning
history_fine_tuning = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=validation_generator,
    class_weight=class_weight_dict,
    callbacks=[early_stopping, reduce_lr]
)


In [ ]:
# Evaluate model on test set
evaluation = model.evaluate(test_generator)
print("Test Accuracy:", evaluation[1])
print("Test Loss:", evaluation[0])

Save model
save_directory = '/content/drive/MyDrive/EM401/models'
os.makedirs(save_directory, exist_ok=True)
model.save(os.path.join(save_directory, 'final_model_class_weights.h5'))

In [ ]:
# Plot training and validation metrics NO FINE TUNE
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()

In [ ]:

# Plot training and validation metrics FINE TUNE
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'] + history_fine_tuning.history['loss'], label='Training Loss (Fine-tuned)')
plt.plot(history.history['val_loss'] + history_fine_tuning.history['val_loss'], label='Validation Loss (Fine-tuned)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'] + history_fine_tuning.history['accuracy'], label='Training Accuracy (Fine-tuned)')
plt.plot(history.history['val_accuracy'] + history_fine_tuning.history['val_accuracy'], label='Validation Accuracy (Fine-tuned)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Predictions
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# True labels
true_classes = test_generator.classes

# Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

print(cm)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=test_generator.class_indices, yticklabels=test_generator.class_indices)
plt.title("Confusion Matrix")
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(true_classes, predicted_classes, target_names=list(test_generator.class_indices.keys())))


In [ ]:
# @title Predictions for validation as well
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predictions
predictions = model.predict(validation_generator)
predicted_classes = np.argmax(predictions, axis=1)

# True labels
true_classes = validation_generator.classes

# Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=validation_generator.class_indices, yticklabels=validation_generator.class_indices)
plt.title("Confusion Matrix")
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()
